<a href="https://colab.research.google.com/github/Prakhar-Gupta-03/CV_Assignments/blob/main/A1/CV_A1_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# plt.ion()   # interactive mode

In [ ]:
# mounting the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Changing the path to the dataset location
%cd drive/MyDrive/Cropped_final
!ls

/content/drive/.shortcut-targets-by-id/1f0rnpZNPfoZGwTlZroaxBVJTf9EtdNNc/Cropped_final
amur_leopard  birds	  brown_bear  people	sika_deer
amur_tiger    black_bear  dog	      roe_deer	wild_boar
